# Age, Cholesterol, Blood Pressure, and Chest Pain in Determining Heart Disease

Andy Hong, Celine Li, Jason Shi, Ibrahim Saeed

For our data analysis, we will only analyze the predictor variables age, chestpain, resting blood pressure and cholestrol level. This is because based on our background knowledge around heart disease, (and for the sake of time and computational power when we use forward selection to determine the best predictors) we believe these variables are most relevant when it comes to determining whether a patient has heart diesease.   

# Preliminary Data Analysis

In [ ]:
# Here we load the packages necessary to run our functions
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Preliminary Exploratory Data Analysis

In the code below, we can prove that the dataset can be read from the web. We will be naming the dataset "heart_data". 


In [ ]:
heart_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
                       col_names = FALSE)
heart_data

We notice that there are no column names for the data set thus we will need to add column names based on what each column represents.

In [ ]:
names <- c("age","sex","cp","trestbps",
           "chol","fbs","restecg","thalach",
           "exang","oldpeak","slope",
           "ca","thal","num")

colnames(heart_data) <- names
heart_data

Let's select for the columns we are interested in and tidy it up. Note that the "num" column tells us the prescence of heart disease within a patient, with a value of 0 meaning no heart disease present and a value of 1 meaning heart disease is present with greater values representing how significant the prescence is all the way up to the maximum value of 4. 

In [ ]:
heart_data_select <- select(heart_data, age, cp, trestbps, chol, num) 
slice(heart_data_select,1:5)


Each row is one unique obsevation and each cell only contains one value. But we are only interested in whether a patient has heart diesease or not. Let's transform the "num" column and make a new column called "diag" of type factor  (abbreviation for diagnosis) so that values equal to 0 translate to "FALSE" which means no heart disease is present whereas any values higher than 0 gets the value "TRUE" which means heart diesease is present. 

In [ ]:
heart_clean <- heart_data_select |>
               mutate(diag = factor(ifelse(is.na(num), NA,  (num > 0)))) |>
               select(-num)

heart_clean

heart_clean_predSel <- heart_data |>
                       mutate(diag = factor(ifelse(is.na(num), NA,  (num > 0)))) |>
                       select(-num,-sex,-thal,-ca)
heart_clean_predSel



Now let's check to see if any missing values. To do this, we can inbed the is.na() function into the sum() function. 

In [ ]:
sum(is.na(heart_data_select))

The value 0 tells us that there are no missing values in our data frame. 

Let's separate our training data from the testing data, set a seed, and summarize the training idea to get a good idea of what we are working with.

In [ ]:
#splitting dataframe into training, testing datasets
set.seed(1)

heart_split <- initial_split(heart_clean, prop = 3/4, strata = diag)
heart_training <- training(heart_split)
heart_testing <- testing(heart_split)

heart_clean

We then can summarize this data into a table below.

In [ ]:
#summarizing training data into a table and getting mean, min, max
heart_summarized <- heart_training %>%
group_by(diag) %>%
summarize(min_age = min(age), max_age = max(age), mean_age = mean(age),
          min_cp  = min(cp), max_cp = max(cp), mean_cp = mean(cp),
          min_chol = min(chol), max_chol = max(chol), mean_chol = mean(chol),
          min_trestbps = min(trestbps), max_trestbps = max(trestbps), 
                             mean_trestbps = mean(trestbps), no_patients = n())
heart_summarized
          
          


We see that there are no anomalies and the number of observations with respect to the diagnosis value (whether or not the patent has heart diesease) is fair. "no_patients" tells us the number of rows. 

In [ ]:
# #Plotting this data to find correlation of diagnosis
# options(repr.plot.width = 8, repr.plot.height = 7)

# # RColorBrewer features for color blindness 
# library(RColorBrewer)


# graph <- ggplot(heart_training, aes(x = age, fill = diag, color = diag)) +
#                  geom_histogram() +
#                  labs(x = "Age (years)", y = "Number of patients", color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Age") +
#                  theme(text = element_text(size = 15)) +
#                  scale_fill_brewer(palette = "Dark2") 

# graph2 <- ggplot(heart_training, aes(x = cp, color = diag, fill = diag)) +
#                  geom_histogram(bins = 15) +
#                  labs(x = "Chest Pain", y = "Number of patients", color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Chest Pains") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph3 <- ggplot(heart_training, aes(x = trestbps, color = diag, fill = diag)) +
#                  geom_histogram() +
#                  labs(x = "Age (years)", y = "Number of patients",
#                       color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Resting Blood Pressure") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph4 <- ggplot(heart_training, aes(x = chol,  color = diag, fill = diag)) +
#                  geom_histogram() +
#                  labs(x = "Cholesterol (mg/dl))", y = "Number of patients", 
#                       color = "Diagnosis", fill = "Diagnosis", title = "Distribution of Cholesterol levels") +
#                  theme(text = element_text(size = 15))+
#                  scale_fill_brewer(palette = "Dark2") 

# graph
# graph2
# graph3
# graph4

# plot_grid(priciest_plot, avocado_plot, n = 1, 
#           labels = c("BuffaloRochester Avocado prices Average vs Total Volume (2018)",
#                      "Houston Avocado prices Average vs Total Volume (2018)"))

-notes to self for this final project

-use a func to align the histograms side by side; there was a func to do this but i forgot but go check 
-use the foward output algorithm to find the num of best predictor variables in ch.6.8 within the textbook
-also they open on tues so can also go tuesday start 3:30 to questions


## Predictor Variable Selection Analysis 

To figure out the best predictor variables to use for our classifier, we will be using the Foward Selection method. 
First, let's select for the predictors we are working with and get the column names of our predictor variables. 

In [ ]:
heart_sub <- heart_clean_predSel |>
             select(diag, 
                    c("age","cp","trestbps",
           "chol","fbs","restecg","thalach",
           "exang","oldpeak","slope",
           "diag") )
pred_names <- colnames(heart_sub |> select(-diag))
heart_sub

Now let's create a model formula using the paste function. We will be naming this heart_formula.

In [ ]:
heart_formula <- paste("diag", "~", paste(pred_names, collapse = "+"))
heart_formula

Lastly, we will create the code to find the best predictors for our purposes. 

(Note: Scaffolding of code below provided from datasciencebook.ca under chapter 6.8. This is cited properly in APA format under references)

In [ ]:
set.seed(1) 
# Empty tibble to store our results
accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

# Here, we create the model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

# Here, we use a 5-fold cross validation for the sake of time and computational power 
heart_vfold <- vfold_cv(heart_sub, v = 5, strata = diag)

# Stores number of total predictors
n_total <- length(pred_names)

# Stores the selected predictors
selected <- c()

# Iterates from 1 to the total number of predictors
# Outside part of nested for loop 
for (i in 1:n_total) {
    
    
# Iterates until every predictor is added 
#Inside part of nested for loop 
    
    accs <- list()
    models <- list()
    for (j in 1:length(pred_names)) {
        
        # create a model string for this combination of predictors
        preds_new <- c(selected, pred_names[[j]])
        model_string <- paste("diag", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        heart_recipe <- recipe(as.formula(model_string), 
                                data = heart_sub) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        # Here, we tune the classifier with our chose predictors
        # and collect the estimated accuracy
        
        acc <- workflow() |>
          add_recipe(heart_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = heart_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        
        # Adding the results to the dataframe
        
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, pred_names[[jstar]])
    pred_names <- pred_names[-jstar]
}
accuracies


Let's plot the number of predictors against the estimated accuracy to see which group of predictors let us obtain the highest accuracy.

In [ ]:
best_pred_plot <- accuracies |>
                  ggplot( aes(x = size, y = accuracy)) +
                  geom_point() +
                  geom_line() +
                  labs(x = "Number of Predictors", y = "Estimated Accuracy") +
                  theme(text = element_text(size = 16)) +
                  scale_x_continuous(breaks = seq(0, 10, by = 1))
best_pred_plot

From this plot, we see that when we use these *"8"* predictors: (Type all the predicors here), we obtain the highest accuracy and thus for our classification problem these are the predictor variables that we shall use when building our model. 



(Note: Above I used all the variables except sex, ca, and thal; result took incredibly long to compute though; originally I used age,cp,trestbps, and chol and on highest obtained a 76% accuracy; here is a screenshot link to see my results originally: https://prnt.sc/zeWsy5Jv6TaY ; latter result took much shorter to run but still around ~1-2min)

